In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from collections import deque
import numpy as np
import multiprocessing
import time
from numpy import array
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
torch.manual_seed(1)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
samplesize = 5

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
ppg_data = pd.read_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/ppg_offset.csv')
ecg_data = pd.read_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/ecg.csv')
offset_data = pd.read_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/offset.csv')

In [ ]:
offset_data


,Unnamed: 0,0
0,0,-35
1,1,24
2,2,21
3,3,1
4,4,40
...,...,...
191233,191233,28
191234,191234,19
191235,191235,-44
191236,191236,27


In [ ]:
ppg_data.iloc[[1]].to_numpy().T[1:].shape

(1000, 1)

In [ ]:
ppg_data.iloc[[1]].to_numpy()[0][1:].T

array([0.33993, 0.34359, 0.34701, 0.35043, 0.35385, 0.36117, 0.36484,
       0.37411, 0.37411, 0.38584, 0.39219, 0.39219, 0.39219, 0.39219,
       0.41978, 0.43028, 0.43028, 0.43028, 0.43028, 0.43053, 0.42759,
       0.42759, 0.4254 , 0.42247, 0.41905, 0.41172, 0.40855, 0.40464,
       0.40073, 0.3939 , 0.39121, 0.38901, 0.38632, 0.38657, 0.38803,
       0.39096, 0.39585, 0.41538, 0.43077, 0.44982, 0.61661, 0.61661,
       0.61661, 0.61661, 0.61661, 0.61661, 0.61661, 0.61661, 0.61661,
       0.62247, 0.62369, 0.62369, 0.62247, 0.61538, 0.61001, 0.60415,
       0.5978 , 0.58364, 0.57582, 0.56777, 0.55922, 0.54115, 0.53187,
       0.52308, 0.50696, 0.50061, 0.49524, 0.49011, 0.48571, 0.4779 ,
       0.47399, 0.46984, 0.46496, 0.45299, 0.44615, 0.44005, 0.42808,
       0.42173, 0.41563, 0.41001, 0.40049, 0.39658, 0.39292, 0.3895 ,
       0.38657, 0.38169, 0.37998, 0.37778, 0.37534, 0.37094, 0.36923,
       0.36801, 0.36752, 0.3685 , 0.36947, 0.37094, 0.37387, 0.37485,
       0.37534, 0.37

In [ ]:
ppg_data.dropna()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1,1,0.33993,0.34359,0.34701,0.35043,0.35385,0.36117,0.36484,0.37411,0.37411,0.38584,0.39219,0.39219,0.39219,0.39219,0.41978,0.43028,0.43028,0.43028,0.43028,0.43053,0.42759,0.42759,0.42540,0.42247,0.41905,0.41172,0.40855,0.40464,0.40073,0.39390,0.39121,0.38901,0.38632,0.38657,0.38803,0.39096,0.39585,0.41538,0.43077,...,0.56117,0.56337,0.56532,0.56679,0.56947,0.57070,0.57167,0.57289,0.57436,0.57778,0.57949,0.58120,0.58291,0.58486,0.58486,0.58486,0.58462,0.58242,0.58144,0.58071,0.57998,0.57998,0.58046,0.58120,0.58217,0.58559,0.58926,0.59267,0.59536,0.59634,0.59560,0.59536,0.59560,0.59731,0.59829,0.60147,0.60611,0.61758,0.62369,0.62882
2,2,0.37485,0.37534,0.37558,0.37534,0.37534,0.37558,0.37607,0.37680,0.37875,0.37998,0.37998,0.38242,0.38437,0.38535,0.38535,0.38535,0.38535,0.38559,0.38584,0.38608,0.38584,0.38535,0.38486,0.38413,0.38217,0.38144,0.38095,0.38095,0.38169,0.38266,0.38388,0.38559,0.38999,0.39267,0.39609,0.40024,0.41026,0.41661,0.42271,...,0.30110,0.30037,0.29988,0.29963,0.29963,0.29963,0.29963,0.30012,0.30183,0.30379,0.30647,0.31136,0.31502,0.31893,0.32283,0.32698,0.33358,0.33651,0.34066,0.34603,0.36093,0.36874,0.37582,0.38803,0.39170,0.39365,0.39365,0.39316,0.39072,0.38901,0.38706,0.38559,0.38510,0.38462,0.38364,0.38046,0.37900,0.37729,0.37485,0.37070
3,3,0.48303,0.48205,0.48107,0.48010,0.47766,0.47619,0.47473,0.47302,0.46960,0.46740,0.46471,0.45934,0.45641,0.45372,0.45128,0.44908,0.44542,0.44371,0.44274,0.44249,0.44396,0.44591,0.44908,0.45275,0.46056,0.46422,0.46691,0.46935,0.46935,0.46789,0.46520,0.46520,0.45690,0.44640,0.44640,0.44640,0.44640,0.42442,0.42442,...,0.33382,0.33504,0.33675,0.33822,0.33846,0.33748,0.33578,0.33455,0.33358,0.33333,0.33333,0.33333,0.33333,0.33333,0.33382,0.33675,0.33871,0.34066,0.34286,0.34554,0.35092,0.35311,0.35531,0.35751,0.36044,0.36215,0.36410,0.36801,0.36899,0.36947,0.36972,0.36899,0.36557,0.36361,0.36166,0.35971,0.35287,0.34994,0.34750,0.34505
4,4,0.52723,0.52015,0.51697,0.51429,0.50965,0.50769,0.50598,0.50452,0.50330,0.50134,0.50012,0.49890,0.49768,0.49451,0.49280,0.49084,0.48669,0.48425,0.48230,0.48059,0.47839,0.47326,0.47033,0.46716,0.46374,0.45714,0.45446,0.45226,0.44982,0.44957,0.44933,0.44957,0.44982,0.45079,0.45128,0.45177,0.45226,0.45324,0.45372,...,0.46716,0.46593,0.46398,0.46154,0.45934,0.45641,0.45568,0.45592,0.45641,0.45983,0.46227,0.46471,0.46667,0.46838,0.46862,0.46886,0.46886,0.46886,0.46886,0.46886,0.46862,0.46740,0.46667,0.46520,0.46325,0.45958,0.45836,0.45739,0.45665,0.45617,0.45665,0.45812,0.45812,0.46007,0.46105,0.46105,0.46105,0.46105,0.46593,0.46691
5,5,0.51966,0.51893,0.51722,0.51624,0.51624,0.51306,0.51306,0.51306,0.50916,0.50916,0.50916,0.50916,0.50110,0.50110,0.49915,0.49841,0.49744,0.49573,0.49499,0.49402,0.49328,0.49206,0.49182,0.49158,0.49182,0.49231,0.49231,0.49206,0.49133,0.48816,0.48571,0.48352,0.48107,0.47473,0.47131,0.46813,0.46496,0.45958,0.45714,...,0.47326,0.47057,0.46960,0.46886,0.46862,0.46886,0.47057,0.47204,0.47375,0.47668,0.47790,0.47863,0.47912,0.47937,0.47937,0.47912,0.47692,0.47692,0.47521,0.47497,0.47473,0.47668,0.47912,0.48230,0.48571,0.48889,0.49353,0.49573,0.49744,0.50012,0.50085,0.50159,0.51038,0.52357,0.53162,0.52967,0.52601,0.52186,0.51306,0.50916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191232,191232,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012

In [ ]:
ecg_data.dropna()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,0,-0.305,-0.310,-0.325,-0.325,-0.330,-0.355,-0.385,-0.395,-0.395,-0.380,-0.425,-0.450,-0.435,-0.460,-0.460,-0.370,-0.300,-0.270,-0.235,-0.230,-0.240,-0.185,-0.165,-0.125,-0.055,-0.065,-0.050,0.025,0.115,0.235,0.325,0.325,0.290,0.190,-0.005,-0.005,-0.005,-0.005,-0.005,...,0.010,0.000,-0.025,-0.045,-0.075,-0.100,-0.115,-0.125,-0.125,-0.125,-0.125,-0.125,-0.160,-0.145,0.115,0.735,1.105,0.675,0.010,-0.115,-0.140,-0.125,-0.120,-0.110,-0.095,-0.075,-0.040,-0.025,-0.005,0.010,0.010,0.020,0.035,0.050,0.060,0.080,0.105,0.135,0.165,0.195
1,1,-0.230,-0.210,-0.210,-0.260,-0.295,-0.215,-0.180,-0.125,-0.015,-0.095,-0.160,-0.125,-0.040,-0.135,-0.135,0.000,0.020,0.040,0.020,-0.010,0.055,0.055,0.030,0.025,0.035,0.040,0.115,0.215,0.265,0.310,0.340,0.330,0.330,0.355,0.415,0.415,0.415,0.415,0.505,...,0.040,0.035,0.010,-0.025,-0.080,-0.105,-0.115,-0.115,-0.115,-0.115,-0.105,-0.105,-0.145,-0.160,0.020,0.705,1.055,0.745,0.020,-0.125,-0.135,-0.120,-0.125,-0.110,-0.085,-0.065,-0.035,-0.015,-0.005,0.000,0.000,0.005,0.005,0.020,0.035,0.070,0.090,0.105,0.130,0.160
2,2,0.110,0.110,0.130,0.145,0.125,0.115,0.105,0.115,0.125,0.135,0.135,0.145,0.155,0.160,0.165,0.160,0.165,0.185,0.195,0.195,0.210,0.215,0.215,0.225,0.225,0.195,0.140,0.090,0.040,-0.005,-0.015,-0.015,-0.030,-0.025,-0.035,-0.035,-0.035,-0.035,0.705,...,-0.130,-0.135,-0.140,-0.130,-0.120,-0.125,-0.135,-0.165,-0.015,0.515,1.005,0.985,0.330,-0.070,-0.105,-0.095,-0.090,-0.085,-0.075,-0.060,-0.035,-0.010,0.005,0.020,0.030,0.045,0.065,0.080,0.100,0.125,0.140,0.155,0.180,0.210,0.225,0.235,0.235,0.225,0.195,0.165
3,3,-0.220,-0.190,-0.165,-0.160,-0.170,-0.175,-0.180,-0.160,-0.140,-0.195,-0.240,-0.245,-0.235,-0.235,-0.215,-0.190,-0.190,-0.195,-0.205,-0.215,-0.185,-0.185,-0.275,-0.370,-0.415,-0.400,-0.370,-0.340,-0.325,-0.350,-0.360,-0.055,-0.055,0.935,0.410,0.410,0.410,0.410,-0.325,...,-0.110,-0.110,-0.105,-0.115,-0.155,-0.015,0.365,1.065,0.980,0.340,-0.100,-0.125,-0.115,-0.115,-0.115,-0.105,-0.075,-0.065,-0.035,-0.005,0.010,0.010,0.030,0.040,0.040,0.070,0.090,0.110,0.130,0.150,0.185,0.195,0.210,0.215,0.210,0.205,0.180,0.135,0.065,0.015
4,4,-0.070,-0.075,-0.080,-0.075,-0.065,-0.055,-0.035,-0.015,-0.015,0.020,0.050,0.045,0.005,-0.020,-0.045,-0.075,-0.090,-0.085,-0.080,-0.080,-0.085,-0.090,-0.105,-0.105,0.080,0.805,1.115,0.720,0.065,-0.065,-0.060,-0.060,-0.065,-0.060,-0.040,-0.040,-0.040,-0.040,-0.040,...,-0.130,-0.160,-0.125,0.235,0.975,1.010,0.420,-0.045,-0.135,-0.115,-0.110,-0.105,-0.095,-0.085,-0.065,-0.035,-0.005,0.005,0.020,0.035,0.050,0.070,0.080,0.095,0.125,0.145,0.165,0.190,0.205,0.215,0.215,0.205,0.195,0.175,0.115,0.070,0.015,-0.045,-0.095,-0.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191232,191232,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,...,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275
191233,191233,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,0.275,...,0.275,0.275,0.275,0.2

In [ ]:
ecg_data.shift(2, axis = 1, fill_value=0.0) 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.070,-0.075,-0.065,-0.055,-0.050,-0.045,-0.035,-0.040,-0.040,-0.040,-0.040,-0.045,-0.045,-0.045,-0.045,-0.045,-0.035,-0.015,0.005,0.020,0.030,0.060,0.060,0.055,0.010,0.010,-0.030,-0.030,-0.030,...,-0.105,-0.105,-0.095,-0.095,-0.090,-0.080,-0.075,-0.070,-0.070,-0.065,-0.065,-0.065,-0.060,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.060,-0.060,-0.060,-0.055,-0.035,-0.015,0.000,0.010,0.030,0.050,0.045,0.015,-0.025,-0.050,-0.075,-0.095,-0.095,-0.100,-0.105,-0.105,-0.100
1,0.0,0.0,-0.045,-0.040,-0.025,-0.005,0.005,0.025,0.050,0.090,0.120,0.155,0.185,0.225,0.250,0.260,0.260,0.245,0.225,0.180,0.120,0.050,-0.015,-0.075,-0.130,-0.150,-0.170,-0.170,-0.175,-0.170,-0.170,-0.170,-0.165,-0.160,-0.160,-0.160,-0.160,-0.160,-0.160,-0.160,...,-0.100,-0.095,-0.095,-0.090,-0.085,-0.080,-0.065,-0.035,-0.025,-0.005,0.030,0.040,0.030,0.000,-0.030,-0.065,-0.095,-0.115,-0.120,-0.125,-0.125,-0.120,-0.115,-0.105,-0.140,-0.155,0.015,0.565,1.140,0.950,0.310,-0.030,-0.075,-0.085,-0.105,-0.115,-0.115,-0.105,-0.100,-0.085
2,0.0,0.0,-0.095,-0.090,-0.085,-0.075,-0.065,-0.050,-0.040,-0.030,-0.015,-0.005,0.005,0.030,0.050,0.080,0.115,0.155,0.185,0.225,0.250,0.265,0.275,0.275,0.255,0.225,0.175,0.105,0.050,-0.010,-0.080,-0.115,-0.145,-0.145,-0.165,-0.170,-0.165,-0.165,-0.165,-0.165,...,-0.160,-0.160,-0.160,-0.160,-0.155,-0.155,-0.155,-0.150,-0.150,-0.150,-0.145,-0.145,-0.145,-0.140,-0.135,-0.135,-0.130,-0.125,-0.120,-0.105,-0.085,-0.055,-0.040,-0.025,-0.005,0.005,-0.005,-0.025,-0.060,-0.095,-0.120,-0.135,-0.140,-0.140,-0.135,-0.130,-0.120,-0.125,-0.170,-0.135
3,0.0,0.0,-0.115,-0.130,-0.130,-0.125,-0.115,-0.115,-0.115,-0.115,-0.115,-0.115,-0.120,-0.125,-0.125,-0.130,-0.125,-0.125,-0.115,-0.105,-0.100,-0.095,-0.085,-0.080,-0.075,-0.075,-0.075,-0.070,-0.065,-0.065,-0.060,-0.055,-0.045,-0.045,0.005,0.015,0.035,0.055,0.055,0.055,...,-0.115,-0.100,-0.075,-0.050,-0.040,-0.020,0.010,0.010,-0.005,-0.045,-0.085,-0.105,-0.125,-0.130,-0.130,-0.125,-0.115,-0.115,-0.105,-0.125,-0.155,-0.015,0.515,1.110,0.920,0.210,-0.020,-0.095,-0.100,-0.110,-0.115,-0.115,-0.110,-0.100,-0.085,-0.080,-0.065,-0.050,-0.035,-0.020
4,0.0,0.0,0.045,0.075,0.115,0.145,0.185,0.225,0.240,0.255,0.255,0.245,0.215,0.160,0.110,0.040,-0.035,-0.080,-0.130,-0.155,-0.170,-0.170,-0.170,-0.175,-0.170,-0.160,-0.160,-0.155,-0.150,-0.145,-0.145,-0.145,-0.140,-0.140,-0.140,-0.135,-0.135,-0.135,-0.135,-0.135,...,-0.135,-0.135,-0.135,-0.135,-0.130,-0.125,-0.125,-0.125,-0.125,-0.120,-0.115,-0.115,-0.120,-0.115,-0.115,-0.115,-0.105,-0.095,-0.095,-0.085,-0.080,-0.075,-0.075,-0.075,-0.070,-0.065,-0.065,-0.065,-0.055,-0.040,-0.015,0.000,0.010,0.040,0.050,0.050,0.015,-0.010,-0.045,-0.075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9482,0.0,0.0,-0.065,-0.055,-0.045,-0.035,-0.030,-0.025,-0.020,-0.010,-0.015,-0.025,-0.040,-0.055,-0.075,-0.085,-0.090,-0.100,-0.105,-0.105,-0.105,-0.105,-0.100,-0.095,-0.085,-0.085,-0.075,-0.070,-0.070,-0.070,-0.065,-0.065,-0.065,-0.065,-0.065,-0.055,-0.035,-0.015,-0.005,0.000,...,-0.055,-0.045,-0.035,-0.025,-0.015,-0.005,0.005,0.010,0.020,0.020,0.005,-0.015,-0.035,-0.055,-0.065,-0.075,-0.085,-0.090,-0.095,-0.095,-0.090,-0.085,-0.085,-0.085,-0.075,-0.075,-0.065,-0.065,-0.065,-0.065,-0.060,-0.060,-0.055,-0.060,-0.060,-0.065,-0.065,-0.055,-0.035,-0.015
9483,0.0,0.0,0.020,0.015,0.005,-0.020,-0.050,-0.080,-0.090,-0.095,-0.095,-0.090,-0.080,-0.075,-0.075,-0.075,-0.075,-0.075,-0.065,

In [ ]:
def prepareData(ppg, ecg):
    if ppg.shape[0] != ecg.shape[0]:
        raise Exception("Data shape doesn't match")
    count = ppg.shape[0]
    train_data, labels = [], []
    for i in range(count):
        offset = random.randint(-5,5)
        # print((array(ecg.iloc[[i]].to_numpy().T)))
        train_data.append(torch.FloatTensor(np.hstack((array(ecg.iloc[[i]].to_numpy()[0][1:].T),array(ppg.iloc[[i]].shift(offset, axis = 1, fill_value=0.0).to_numpy()[0][1:].T)))))
        labels.append(torch.FloatTensor([offset]))
    train_length =  181676
    return train_data[:train_length], labels[:train_length], train_data[train_length:], labels[train_length:]
train_data, train_label, test_data, test_label = prepareData(ppg_data, ecg_data)

In [ ]:
def prepareMovingData(ppg, ecg):
    if ppg.shape[0] != ecg.shape[0]:
        raise Exception("Data shape doesn't match")
    count = ppg.shape[0]
    train_data, labels = [], []
    for i in range(count):
        if ppg.iloc[[i]].isnull().values.any() or ecg.iloc[[i]].isnull().values.any():
            continue
        offset=offset_data.iloc[[i]].to_numpy()[0][1]
            # print((array(ecg.iloc[[i]].to_numpy().T)))
        train_data.append(torch.FloatTensor(np.hstack((array(ecg.iloc[[i]].to_numpy().T[1:]),array(ppg.iloc[[i]].to_numpy().T[1:])))))
        labels.append(torch.FloatTensor([offset]))
    train_length =  181676
    return train_data[:train_length], labels[:train_length], train_data[train_length:], labels[train_length:]
train_data, train_label, test_data, test_label = prepareMovingData(ppg_data, ecg_data)


In [ ]:
train_data.shape

AttributeError: ignored

In [3]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, data, labels):
        self.labels = labels
        self.data = data

  def __len__(self):
        return len(self.data)

  def __getitem__(self, index):
        X = self.data[index]
        y = self.labels[index]

        return X, y

In [ ]:
train_label

[tensor([-35.]),
 tensor([24.]),
 tensor([21.]),
 tensor([1.]),
 tensor([40.]),
 tensor([27.]),
 tensor([-46.]),
 tensor([-35.]),
 tensor([19.]),
 tensor([5.]),
 tensor([-17.]),
 tensor([3.]),
 tensor([31.]),
 tensor([0.]),
 tensor([1.]),
 tensor([28.]),
 tensor([32.]),
 tensor([49.]),
 tensor([-10.]),
 tensor([17.]),
 tensor([25.]),
 tensor([34.]),
 tensor([-30.]),
 tensor([-11.]),
 tensor([-18.]),
 tensor([43.]),
 tensor([-30.]),
 tensor([18.]),
 tensor([-47.]),
 tensor([4.]),
 tensor([-12.]),
 tensor([7.]),
 tensor([-1.]),
 tensor([21.]),
 tensor([-46.]),
 tensor([4.]),
 tensor([-4.]),
 tensor([20.]),
 tensor([0.]),
 tensor([11.]),
 tensor([-4.]),
 tensor([34.]),
 tensor([-7.]),
 tensor([16.]),
 tensor([15.]),
 tensor([-36.]),
 tensor([-18.]),
 tensor([-24.]),
 tensor([43.]),
 tensor([-42.]),
 tensor([38.]),
 tensor([46.]),
 tensor([48.]),
 tensor([-15.]),
 tensor([30.]),
 tensor([-42.]),
 tensor([46.]),
 tensor([-3.]),
 tensor([-28.]),
 tensor([4.]),
 tensor([10.]),
 tensor([-42.])

In [ ]:
#prepare data first time
traindataset = CustomDataset(train_data,train_label)
# val_data = CustomDataset(test_input,test_label)
testdataset = CustomDataset(test_data,test_label)


In [ ]:
torch.save(traindataset, '/content/drive/My Drive/QueenslandDataset/Processed Data/train_data_moving_final.pt')
torch.save(traindataset, '/content/drive/My Drive/QueenslandDataset/Processed Data/test_data_moving_final.pt')

In [ ]:
traindataset.__getitem__(0)

(tensor([[-0.2300,  0.3399],
         [-0.2100,  0.3436],
         [-0.2100,  0.3470],
         ...,
         [ 0.1050,  0.6176],
         [ 0.1300,  0.6237],
         [ 0.1600,  0.6288]]), tensor([24.]))

In [ ]:
traindataset.__len__()

181676

In [ ]:
testdataset.__len__()

9516

In [4]:
#load from drive
traindataset = torch.load('/content/drive/My Drive/QueenslandDataset/Processed Data/train_data_moving_final.pt')
testdataset = torch.load('/content/drive/My Drive/QueenslandDataset/Processed Data/test_data_moving_final.pt')


In [ ]:
!ls '/content/drive/My Drive/QueenslandDataset/Processed Data'

filtered_ecg.csv	  test_data_with_other_data.pt
filtered_hr_sys_dias.csv  train_data.pt
filtered_ppg.csv	  train_data_with_other_data.pt
test_data.pt


In [5]:
train_batch_size = 44
test_batch_size = 39
train_loader = DataLoader(traindataset, shuffle=True, batch_size=train_batch_size)
test_loader = DataLoader(testdataset, shuffle=True, batch_size=test_batch_size)

In [ ]:
testdataset.__len__()

9516

In [ ]:
traindataset.__len__()

181676

In [6]:
class LSTM(nn.Module):
    def __init__(self, input_size=2, output_size = 1, hidden_dim = 512, n_layers = 3, drop_prob=0):
        super(LSTM, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        # self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        # x = x.long()
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # lstm_out = self.dropout(lstm_out)
        out = self.fc(lstm_out)
        # out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        # print(out.shape)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [7]:
model = LSTM()
model.to(device)

lr = 0.0001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [ ]:
epochs = 10
counter = 0
print_every = 500
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    # print(batch_size)
    print("Epoch {}/{}".format(i+1, epochs))
    print('-' * 20)
    epoch_loss = 0.0
    h = model.init_hidden(train_batch_size)
    # model.zero_grad()
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)

        output, h = model(inputs, h)
        # print("a",output.shape)
        # print("b",labels.squeeze().shape)
        optimizer.zero_grad()
        loss = criterion(output, labels.squeeze())
        # print(output)
        # print(labels)
        # break
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += output.shape[0] * loss.item()
        if counter%print_every == 0:
        #     # print(batch_size)
        #     val_h = model.init_hidden(batch_size)
        #     val_losses = []
        #     model.eval()
        #     for inp, lab in val_loader:
        #         val_h = tuple([each.data for each in val_h])
        #         inp, lab = inp.to(device), lab.to(device)
        #         out, val_h = model(inp, val_h)
        #         val_loss = criterion(out, lab.squeeze())
        #         val_losses.append(val_loss.item())
                
            # # model.train()
            # print(inputs)
            # print(labels)
            # print(output)
            print("Epoch: {}/{}".format(i+1, epochs),
                  "Step: {}".format(counter),
                  "Loss: {:.6f}".format(loss.item()))
            #       "Val Loss: {:.6f}".format(np.mean(val_losses)))
            # if np.mean(val_losses) <= valid_loss_min:
            #     # torch.save(model.state_dict(), './state_dict.pt')
            #     print('Validation loss decreased ({:.6f} --> {:.6f}).'.format(valid_loss_min,np.mean(val_losses)))
            #     valid_loss_min = np.mean(val_losses)
    # break
    print("Epoch {} Loss {}".format(i+1, epoch_loss / traindataset.__len__()))

Epoch 1/10
--------------------
Epoch: 1/10 Step: 500 Loss: 832.508362
Epoch: 1/10 Step: 1000 Loss: 735.459656
Epoch: 1/10 Step: 1500 Loss: 772.799500
Epoch: 1/10 Step: 2000 Loss: 920.008545
Epoch: 1/10 Step: 2500 Loss: 734.665588
Epoch: 1/10 Step: 3000 Loss: 834.039062
Epoch: 1/10 Step: 3500 Loss: 1040.166992
Epoch: 1/10 Step: 4000 Loss: 557.169006
Epoch 1 Loss 833.8373938144971
Epoch 2/10
--------------------
Epoch: 2/10 Step: 4500 Loss: 690.996277
Epoch: 2/10 Step: 5000 Loss: 871.907349
Epoch: 2/10 Step: 5500 Loss: 789.432190
Epoch: 2/10 Step: 6000 Loss: 1156.213501
Epoch: 2/10 Step: 6500 Loss: 737.460510
Epoch: 2/10 Step: 7000 Loss: 968.307556
Epoch: 2/10 Step: 7500 Loss: 881.106445
Epoch: 2/10 Step: 8000 Loss: 895.637634
Epoch 2 Loss 833.8819090274322
Epoch 3/10
--------------------
Epoch: 3/10 Step: 8500 Loss: 762.900757
Epoch: 3/10 Step: 9000 Loss: 871.182190
Epoch: 3/10 Step: 9500 Loss: 926.402832
Epoch: 3/10 Step: 10000 Loss: 588.624634
Epoch: 3/10 Step: 10500 Loss: 1055.41003

In [ ]:
#saving model to drive

torch.save(model.state_dict(), f'/content/drive/My Drive/DummyGaussianData/model_with_2_input.pt')

In [ ]:
#loading model from drive
model = LSTM(samplesize*2)
model.to(device)
model.load_state_dict(torch.load('/content/drive/My Drive/DummyGaussianData/model.pt'))
lr = 0.0001
criterion = nn.MSELoss()

In [ ]:
test_losses = []
num_correct = 0
h = model.init_hidden(test_batch_size)
counter = 0
model.eval()
for inputs, labels in test_loader:
    counter+=1
    print(f'Starting Batch {counter}')
    print('-' * 20)
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    # print(inputs)
    output, h = model(inputs, h)
    test_loss = criterion(output, labels.squeeze())
    test_losses.append(test_loss.item())
    pred = torch.round(output)  # Rounds the output to 0/1
    print(f" Offset {labels.squeeze().data},\n Prediction {output.data},\n Prediction rounded {pred}")
    correct_tensor = pred.eq(labels.squeeze().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

Starting Batch 1
--------------------
 Offset tensor([-25.,  47., -15.,  42.,  31.,  36.,  29., -18.,  30.,  47.,  41., -47.,
         14.,  22.,  -3., -44., -49.,  32.,  37.,   3.,  43.,  21.,  21., -13.,
        -19., -31., -22.,  -1., -40., -16.,  47.,  37., -20., -34.,  46., -22.,
         44.,  -6., -11.,  20., -31.,  35.,  42., -50.,  -3., -27.,  25., -17.,
         20., -31.,  35.,  40.,  26.,   4., -18., -31.,  23.,  35.,  37., -45.,
         31., -35., -36.,   3., -45.,  24.,  43.,  24., -45., -20., -28.,  48.,
        -46.,  32., -29.,  24., -13., -33.,  35., -13.,  12.,  31., -13., -28.,
         28., -26.,  47., -26.,  16.,  13., -37., -21.,   2.,  36.,   9.,  -6.,
         27.,  18., -41.,  43.,  15.,   0., -31., -42., -38.,  24.,   1.,  -8.,
          6.,  48.,  44.,  19.,   2.,  11., -14.,  -4.,  21.,  23.,  47.,  13.,
         21., -10., -34.,  25.,  45., -18.,   7.,  43., -11., -20.,  35., -15.,
          0., -34., -46., -38.,  26.,   8.,  11., -23.,  14., -31., -48., 